In [1]:
import aiohttp
import asyncio
from aiohttp import BasicAuth
from bs4 import BeautifulSoup
from aiohttp.client_exceptions import ContentTypeError
from urllib.parse import urlencode
import re
import json


In [77]:
class User:
    def __init__(self, proxy:str, account:dict):
        ''' Аналог личного кабинета на сайте https://pass.ska.ru/
        В данный момент 1 юзер может купить 5 билетов

        Пример использования 
        1)Инициализация класса"
                proxy = "94.137.90.59:50100@mikheyev9:aXUbosImHR"
                account = {'login':'mikheyev9@gmail.com',
                           'password': 'Ai5F,;baB:rVf!8'} 
                user_1 = User(proxy=proxy,
                              account = account)

        2)Cоздать сессию 
                status_code = await user_1.make_session()

        3)Если status_code == 200:
            аутентифицировать пользователя:
                auth_status = await user_1.auth()

        4)Если auth_status == True:
            Добавить билеты в корзину:
                event_id = '1933'
                ticket_info = {'categoryId': '1',
                                'id': '572366',
                                'name': 'Сектор 615 Ряд 15 Место 118',
                                'orderId': '',
                                'price': '550.00',
                                'quant': '1'}
                post_status = await user_1.put_ticket_to_cart(event_id=event_id, 
                                                              ticket_info=ticket_info)

        5)При необходимости посмотреть какие билеты уже в корзине:
                print(self.all_tickets)

        6)Если в корзине есть билеты выводим на оплату:
                event_id = '1933'
                url_for_payment, account = await x.pay_tickets_from_cart(event_id)
                Получаем ссылка на оплату и аккуант                                                      
            

        *)async def make_session - создать сессию, вернет cатус код запроса на создание
        *)async def auth - аутентификация юзера если все ок вернет True иначе False
        *)async def close_session - закрыть сессию
        *)async def put_ticket_to_cart(event_id:ид мероприятия, ticket_info:билет)
            положить в корзину билет вернет True если OK иначе False

        *)async def pay_tickets_from_cart(event_id:ид мероприятия) - бронирование билетов в системе и
            создание ссылки на оплату. Вернет url оплаты и данные пользователя self.account если ОК,
            иначе raise AssertionError
        '''

        self.proxy_url, self.proxy_auth = self.make_proxy(proxy)
        self.proxy_status = None
        self.auth_status = None
        self.session = None
        self.cookies = {}

        self.account = account
        self.all_tickets = []

        self.headers_post =  {
            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "accept-language": "en-US,en;q=0.9,ru;q=0.8",
            "cache-control": "max-age=0",
            "content-type": "application/x-www-form-urlencoded",
            "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "\"Linux\"",
            "sec-fetch-dest": "document",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "same-origin",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1",
            "Referer": "https://pass.ska.ru/en/auth/?back_domain=tickets&back_url=%2F",
            "Referrer-Policy": "strict-origin-when-cross-origin",
            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }

    @staticmethod
    def make_proxy(proxy:str) -> tuple[str, BasicAuth]:
        #proxy = "94.137.90.59:50100@mikheyev9:aXUbosImHR"
        domain, user_info = proxy.split('@')
        proxy_url = f'http://{domain}'
        proxy_login, proxy_password = user_info.split(':')
        proxy_auth = BasicAuth(login=proxy_login, password=proxy_password)
        return proxy_url, proxy_auth

    async def _init_session(self) -> None:
        if not self.session:
            self.session = aiohttp.ClientSession()

    async def close_session(self) -> None:
        if self.session:
            await self.session.close()
            self.session = None


    async def make_session(self) -> str:
        '''Инициализация сессии и проверка доступности прокси,
           Если все ок, self.proxy_status == 200
        '''

        await self._init_session()
        headers = {
            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "accept-language": "en-US,en;q=0.9,ru;q=0.8",
            "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "\"Linux\"",
            "sec-fetch-dest": "document",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "same-site",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1",
            "Referer": "https://tickets.ska.ru/",
            "Referrer-Policy": "strict-origin-when-cross-origin",
            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        url = 'https://pass.ska.ru/en/auth/?back_domain=tickets&back_url=%2F'
        
        async with self.session.get(url, headers=headers,
                                    proxy=self.proxy_url,
                                    proxy_auth=self.proxy_auth) as response:
            text = await response.text()
            self.CSRF = self._find_csrf(text)
            self.proxy_status = response.status
            print(self.CSRF, 'CSRF token')
            return response.status
    
    @staticmethod
    def _find_csrf(text):
        soup = BeautifulSoup(text, 'lxml')
        meta = soup.find('meta', attrs={'name':'csrf-token'})
        return meta.get('content')
    
    async def auth(self) -> bool:
        '''Авторизация пользователя в ситсеме покупки билетов
            Если все ок, self.auth_status == True

            self.account = { 'login': 'mikheyev9@gmail.com',
                            'password': 'Ai5F,;baB:rVf!8' }
        '''

        login = self.account['login']
        password = self.account['password']
        data = {
            '_csrf': self.CSRF,
            'login': login,
            'password': password,
            'auth_type': 'email',
            'send_submit': 'Y'
        }
        async with self.session.post(url='https://pass.ska.ru/en/auth/', 
                                     proxy=self.proxy_url,
                                     proxy_auth=self.proxy_auth,
                                     headers=self.headers_post, data=data) as response:
            if response.status == 200:
                self.auth_status = True
                print(response, self.auth_status, 'self.auth_status')
                return True
            return False


    async def put_ticket_to_cart(self, event_id:str, ticket_info:dict) -> bool:
        '''Положить билет в корзину пользователя
            Если все ок вернет True иначе False
        '''

        ticket = {
            'operation': 'add',
            'id': ticket_info['id'],
            'categoryId': ticket_info['categoryId'],
            'name': ticket_info['name'],
            'quant': ticket_info['quant'],
            'price': ticket_info['price'],
            'orderId': ''
        }

        async with self.session.post(url=f'https://tickets.ska.ru/cart/{event_id}', 
                                     headers=self.headers_post,
                                     proxy=self.proxy_url,
                                     proxy_auth=self.proxy_auth, 
                                     data=ticket) as response:
            try:
                response = await response.json()
            except ContentTypeError:
                return False
            if response.get('result', False) == True:
                self.all_tickets = response.get('list')
                return True
            elif response.get('result') == False:
                self.all_tickets = response.get('list')
                return False
            raise AssertionError(f'https://tickets.ska.ru/cart/{event_id} something went wrong')
    
    async def _create_order_to_pay(self, event_id:str) -> str | bool:
        '''Предварительный запрос на оплату
           Если все ок вернет id созданного платежа в их(СКА) системе
           Если нет вернет False'''

        pay_method = {'card': 'other'}
        async with self.session.post(url=f'https://tickets.ska.ru/create-order/{event_id}', 
                                    proxy=self.proxy_url,
                                    proxy_auth=self.proxy_auth,
                                    headers=self.headers_post,
                                    data=pay_method) as response:
            try:
                json = await response.json()
            except ContentTypeError:
                return False
            if json.get('result'):
                return json['order']
            return False

    async def pay_tickets_from_cart(self, event_id:str) -> AssertionError | tuple[str, dict]:
            '''Создание запроса на оплату и получение ссылки от эквайринга
            '''
            
            create_order = await self._create_order_to_pay(event_id)
            if not create_order:
                raise AssertionError(f"Cannot pay this tickets {self.all_tickets}")
            pay_method = {'card': 'other'}

            async with self.session.post(url=f'https://tickets.ska.ru/pay-order/{create_order}', 
                                    proxy=self.proxy_url,
                                    proxy_auth=self.proxy_auth,
                                    headers=self.headers_post,
                                    data=pay_method) as response:
                
                text = await response.text()
                with open('TEST.html', 'w', encoding='utf-8') as f:
                    f.write(text)
                soup = BeautifulSoup(text, 'lxml')
                form_gazprombank = soup.find('form', attrs={'action':"https://www.pga.gazprombank.ru:443/pages/"})
                lang = form_gazprombank.find('input', attrs={'name':'lang'}).get('value')
                merch_id = form_gazprombank.find('input', attrs={'name':'merch_id'}).get('value')
                back_url_s = form_gazprombank.find('input', attrs={'name':'back_url_s'}).get('value')
                back_url_f = form_gazprombank.find('input', attrs={'name':'back_url_f'}).get('value')
                order_id = form_gazprombank.find('input', attrs={'name':'o.order_id'}).get('value')
                tickets_count = form_gazprombank.find('input', attrs={'name':'o.tickets_count'}).get('value')
                action_id = form_gazprombank.find('input', attrs={'name':'o.action_id'}).get('value')
                SUBMIT = form_gazprombank.find('input', attrs={'name':'SUBMIT'}).get('value')
                
                

                url = "https://www.pga.gazprombank.ru:443/pages/"
                params = {
                    "lang": lang,
                    "merch_id": merch_id,
                    "back_url_s": back_url_s,
                    "back_url_f": back_url_f,
                    "o.order_id": order_id,
                    "o.tickets_count": tickets_count,
                    "o.action_id": action_id,
                    "SUBMIT": SUBMIT
                }
                encoded_params = urlencode(params)
                full_url = f"https://www.pga.gazprombank.ru:443/pages/?{encoded_params}"

                return full_url, self.account

In [78]:
async def main():
    proxy = "94.137.90.59:50100@mikheyev9:aXUbosImHR"
    account = { 'login': 'mikheyev9@gmail.com',
                'password': 'Ai5F,;baB:rVf!8' }
    x = User(proxy, account)
    check_proxy = await x.make_session()
    print(check_proxy)  # Выведет статус ответа
    return x

In [79]:
x = await main()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f04f8761610>


czcKSJkI3DOT1yrTtYuj0jLWjTMv4FhcHZBjEHzelXQJcVgBylmrZaKBY5Hw_s3geZ77UUe3LQZk-DImBIrXIg== CSRF token
200


In [80]:
await x.auth()

<ClientResponse(https://tickets.ska.ru/) [200 OK]>
<CIMultiDictProxy('Server': 'nginx', 'Date': 'Sun, 18 Feb 2024 15:07:19 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Set-Cookie': 'SKA_SESSION=j2t3op7fpa4js0o8qsclps0i61; expires=Mon, 19-Feb-2024 15:07:15 GMT; Max-Age=86400; path=/', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Strict-Transport-Security': 'max-age=31536000;', 'Content-Encoding': 'gzip')>
 True self.auth_status


In [81]:
tik = {   'categoryId': '1',
    'id': '572366',
    'name': 'Сектор 615 Ряд 15 Место 118',
    'orderId': '',
    'price': '550.00',
    'quant': '1'}
await x.put_ticket_to_cart('1933', tik)

True

In [82]:

print(x.all_tickets)
print(len(x.all_tickets))


[{'id': '572366', 'categoryId': '1', 'name': 'Сектор 615 Ряд 15 Место 118', 'quant': '1', 'price': '550.00', 'type': 0, 'orderId': ''}]
1


In [83]:
res = await x.pay_tickets_from_cart('1933')

In [84]:
print(res)

https://www.pga.gazprombank.ru:443/pages/?lang=ru&merch_id=GX7UWIRPT48ZMJE576NTKMP0ZI8TGS8U&back_url_s=https%3A%2F%2Ftickets.ska.ru%2Fpersonal-thank-you%2F1659963&back_url_f=https%3A%2F%2Ftickets.ska.ru%2Fview-orders-list&o.order_id=1659963&o.tickets_count=1&o.action_id=1933&SUBMIT=%D0%9F%D0%B5%D1%80%D0%B5%D0%B9%D1%82%D0%B8+%D0%BA+%D0%BE%D0%BF%D0%BB%D0%B0%D1%82%D0%B5


In [2]:
###
headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'ru,en;q=0.9',
            'cache-control': 'max-age=0',
            'connection': 'keep-alive',
            'host': 'tickets.ska.ru',
            "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
            'sec-ch-ua-mobile': '?0',
            "sec-ch-ua-platform": "\"Linux\"",
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
url = 'https://tickets.ska.ru/view-available-zones/1933'
async with aiohttp.ClientSession() as session:
    async with session.get(url, headers=headers) as response:
        text = await response.text()
        soup = BeautifulSoup(text, 'lxml')

        total_sector = []

        text_js = soup.find('div', class_='wrp-main').find('script').text
        no_render_data = text_js[text_js.index('['):text_js.index('];')+1]
        dict_data = eval(no_render_data.replace('null', 'None'))

        all_sectors_structure = {}

        for sectors in dict_data:
                sector = sectors.get('name')
                if sector:
                    sector_id = sectors.get('id')
                    url_split = url.split("/")
                    url_sector = f'seats-list/{url_split[-1]}/{sector_id}'
                    url_to_sector = '/'.join(url_split[:-2]) + '/' + url_sector
                    sectors.update({'url':url_to_sector})
                    all_sectors_structure.setdefault(sector, {}).update({
                                                            'url':url_to_sector
                                                            })
                    #print(url_to_sector)

In [3]:
print(dict_data)

[{'id': '560617', 'name': 304, 'quant': '7', 'zoneId': '-99999', 'zoneTypeId': '100', 'price': [1400, 3400], 'url': 'https://tickets.ska.ru/seats-list/1933/560617'}, {'id': '560734', 'name': 305, 'quant': '45', 'zoneId': '-99999', 'zoneTypeId': '100', 'price': [1400, 3400], 'url': 'https://tickets.ska.ru/seats-list/1933/560734'}, {'id': '560955', 'name': 'Ресторан 4 этаж', 'quant': '68', 'zoneId': '328', 'zoneTypeId': '110', 'price': [2400, 2400], 'url': 'https://tickets.ska.ru/seats-list/1933/560955'}]


In [4]:
all_sectors_structure

{304: {'url': 'https://tickets.ska.ru/seats-list/1933/560617'},
 305: {'url': 'https://tickets.ska.ru/seats-list/1933/560734'},
 'Ресторан 4 этаж': {'url': 'https://tickets.ska.ru/seats-list/1933/560955'}}

In [22]:
all_prices_id = {}
def parsing_seat(seat_name):
    #Сектор 304 Ряд 22 Место 107
    if not all(i in seat_name  for i in ('Сектор', 'Ряд', 'Место')):
        return False
    sector = re.search(r'(?<=Сектор) *\d+', seat_name)[0].strip()
    row = re.search(r'(?<=Ряд) *\d+', seat_name)[0].strip()
    place = re.search(r'(?<=Место) *\d+', seat_name)[0].strip()
    return sector, row, place
    
async with aiohttp.ClientSession() as session:
    async with session.get('https://tickets.ska.ru/seats-list/1933/560617', headers=headers) as response:
        json = await response.json()
        prices = json.get('prices')
        for price in prices:
            all_prices_id.setdefault(price["categoryId"], {}).update({
                price["pricezoneId"]: price['value']
            })

        # ticket_info = {'categoryId': '1',
        #                         'id': '572366',
        #                         'name': 'Сектор 615 Ряд 15 Место 118',
        #                         'orderId': '',
        #                         'price': '550.00',
        #                         'quant': '1'}

        all_tickets_structure = {}
        seats = json.get('seats')
        if len(seats) <= 0:
            print(False)
        first_blood = 1 # При первом проходе очищаются все места для этого сектора, чтобы обновить информацию о наличии
        for seat in seats:
            categoryId = seat.get('quant')
            id = seat.get('id')
            name = seat.get('name')
            price_zone = seat.get('pricezoneId')
            price = all_prices_id[categoryId].get(price_zone)
            sector_row_place = parsing_seat(name)
            if sector_row_place:
                sector, row, place = sector_row_place
                all_tickets_structure.setdefault(sector, {})
                if first_blood:
                    all_tickets_structure[sector] = {row: {}}
                    first_blood = 0
                else:
                    all_tickets_structure[sector].setdefault(row, {})
                all_tickets_structure[sector][row].setdefault(place, {}).update({
                                'categoryId': categoryId,
                                'id': id,
                                'name': name,
                                'orderId': '',
                                'price': price,
                                'quant': '1'
                })
            print(sector_row_place)
            
        

('304', '22', '107')
('304', '23', '6')
('304', '23', '10')
('304', '24', '10')
('304', '24', '106')
('304', '25', '3')
('304', '25', '110')


In [20]:
print(all_prices_id)

{'1': {'326': '3400.00', '408': '1400.00'}}


In [23]:
print(all_tickets_structure)

{'304': {'22': {'107': {'categoryId': '1', 'id': '560647', 'name': 'Сектор 304 Ряд 22 Место 107', 'orderId': '', 'price': '3400.00', 'quant': '1'}}, '23': {'6': {'categoryId': '1', 'id': '560657', 'name': 'Сектор 304 Ряд 23 Место 6', 'orderId': '', 'price': '1400.00', 'quant': '1'}, '10': {'categoryId': '1', 'id': '560661', 'name': 'Сектор 304 Ряд 23 Место 10', 'orderId': '', 'price': '3400.00', 'quant': '1'}}, '24': {'10': {'categoryId': '1', 'id': '560686', 'name': 'Сектор 304 Ряд 24 Место 10', 'orderId': '', 'price': '3400.00', 'quant': '1'}, '106': {'categoryId': '1', 'id': '560696', 'name': 'Сектор 304 Ряд 24 Место 106', 'orderId': '', 'price': '3400.00', 'quant': '1'}}, '25': {'3': {'categoryId': '1', 'id': '560704', 'name': 'Сектор 304 Ряд 25 Место 3', 'orderId': '', 'price': '1400.00', 'quant': '1'}, '110': {'categoryId': '1', 'id': '560725', 'name': 'Сектор 304 Ряд 25 Место 110', 'orderId': '', 'price': '3400.00', 'quant': '1'}}}}


In [81]:
###########
class Tickets:
    def __init__(self, url):
        self.url = url #'https://tickets.ska.ru/view-available-zones/1928'

        self.all_free_sectors = []
        self.all_tickets_structure = {}
        self.all_prices_id = {}

        self.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'ru,en;q=0.9',
            'cache-control': 'max-age=0',
            'connection': 'keep-alive',
            'host': 'tickets.ska.ru',
            "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
            'sec-ch-ua-mobile': '?0',
            "sec-ch-ua-platform": "\"Linux\"",
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        
        
    async def update_all_free_sectors(self):
        async with aiohttp.ClientSession() as session:
            async with session.get(self.url, headers=self.headers) as response:
                text = await response.text()
                soup = BeautifulSoup(text, 'lxml')

                text_js = soup.find('div', class_='wrp-main').find('script').text
                no_render_data = text_js[text_js.index('['):text_js.index('];')+1]
                dict_data = eval(no_render_data.replace('null', 'None'))
                #dict_data=[{'id': '560617', 'name': 304, 'quant': '31', 'zoneId': '-99999', 'zoneTypeId': '100',
                #           'price': [1200, 2400], 'url': 'https://tickets.ska.ru/seats-list/1928/560617'}, ... ]

                all_sectors_structure = {}

                for sectors in dict_data:
                        sector = sectors.get('name')
                        if sector:
                            sector_id = sectors.get('id')
                            url_split = url.split("/")
                            url_sector = f'seats-list/{url_split[-1]}/{sector_id}'
                            url_to_sector = '/'.join(url_split[:-2]) + '/' + url_sector
                            sectors.update({'url':url_to_sector})
                            #print(url_to_sector)

                            all_sectors_structure.setdefault(str(sector), {}).update({
                                                            'url':url_to_sector
                                                            })

                self.all_free_sectors = all_sectors_structure
                return dict_data
    
    async def update_rows_seats_in_sector(self, sector_name: str):
        if sector_name not in self.all_free_sectors:
            await self.update_all_free_sectors()
            #print(self.all_free_sectors, 'all_free_sectors')
            if sector_name not in self.all_free_sectors:
                self.all_tickets_structure[sector_name] = {}
                print(f"Sector:{sector_name} isnt availible")
                return False
            
        url = self.all_free_sectors[sector_name]['url']
        async with aiohttp.ClientSession() as session:
            async with session.get(url, headers=self.headers) as response:
                json = await response.json()
                prices = json.get('prices')
                for price in prices:
                    self.all_prices_id.setdefault(price["categoryId"], {}).update({
                        price["pricezoneId"]: price['value']
                    })

                seats = json.get('seats')
                if len(seats) <= 0:
                    return False
                first_blood = 1 # При первом проходе очищаются все места для этого сектора, чтобы обновить информацию о наличии
                for seat in seats:
                    categoryId = seat.get('quant')
                    id = seat.get('id')
                    name = seat.get('name')
                    price_zone = seat.get('pricezoneId')
                    price = self.all_prices_id[categoryId].get(price_zone)
                    sector_row_place = self.parsing_seat(name)
                    if sector_row_place:
                        sector, row, place = sector_row_place
                        self.all_tickets_structure.setdefault(sector, {})
                        if first_blood:
                            self.all_tickets_structure[sector] = {row: {}}
                            first_blood = 0
                        else:
                            self.all_tickets_structure[sector].setdefault(row, {})
                        self.all_tickets_structure[sector][row].setdefault(place, {}).update({
                                        'categoryId': categoryId,
                                        'id': id,
                                        'name': name,
                                        'orderId': '',
                                        'price': price,
                                        'quant': '1'
                        })
        return True
                    

    @staticmethod   
    def parsing_seat(seat_name):
        #Сектор 304 Ряд 22 Место 107
        if not all(i in seat_name  for i in ('Сектор', 'Ряд', 'Место')):
            return False
        sector = re.search(r'(?<=Сектор) *\d+', seat_name)[0].strip()
        row = re.search(r'(?<=Ряд) *\d+', seat_name)[0].strip()
        place = re.search(r'(?<=Место) *\d+', seat_name)[0].strip()
        return sector, row, place         

        
    async def find_place_in_sector(self, sector, row, place):
        update_sector = await self.update_rows_seats_in_sector(sector)
        if not update_sector:
            return False
        if sector in self.all_tickets_structure:
            all_rows = [ i for i in self.all_tickets_structure.get(sector, {})]
            if row in all_rows:
                all_seats = [ i for i in self.all_tickets_structure[sector].get(row, {})]
                if place in all_seats:
                    return self.all_tickets_structure[sector][row].get(place)
        return False



In [82]:
x = Tickets('https://tickets.ska.ru/view-available-zones/1933')

In [84]:
print(x.all_free_sectors)

{'304': {'url': 'https://tickets.ska.ru/seats-list/1933/560617'}, '305': {'url': 'https://tickets.ska.ru/seats-list/1933/560734'}, 'Ресторан 4 этаж': {'url': 'https://tickets.ska.ru/seats-list/1933/560955'}}


In [83]:
res = await x.update_rows_seats_in_sector('304')

In [85]:
print(x.all_tickets_structure['304']['23'])

{'6': {'categoryId': '1', 'id': '560657', 'name': 'Сектор 304 Ряд 23 Место 6', 'orderId': '', 'price': '1400.00', 'quant': '1'}, '10': {'categoryId': '1', 'id': '560661', 'name': 'Сектор 304 Ряд 23 Место 10', 'orderId': '', 'price': '3400.00', 'quant': '1'}}


In [94]:
place_1 = await x.find_place_in_sector('304', '23', '61')

In [93]:
print(place_1)

False


In [8]:
###logic
with open('accounts.json', 'r') as file:
    user_accounts = json.load(file)
print(user_accounts)
with open('proxies.json', 'r') as file:
    proxies = json.load(file)
print(proxies)
with open('need_tikets.json', 'r') as file:
    need_tikets = json.load(file)
print(need_tikets)


[{'login': 'mikheyev9@gmail.com', 'password': 'Ai5F,;baB:rVf!8'}, {'login': 'mikheyev98@gmail.com', 'password': 'Ai5F,;baB:rVf!8'}]
['94.137.90.59:50100@mikheyev9:aXUbosImHR', '94.137.90.59:50100@mikheyev9:aXUbosImHR']
{'207': {'30': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 111, 112, 113], '28': [105, 106, 109, 110, 111, 112]}, '205': {'19': [4, 5, 6, 7, 8, 9, 10], '22': [105, 106, 107, 108, 109]}}


In [10]:
def dict_to_tuples(dict_data):
  tuples = []
  for outer_key, inner_dict in dict_data.items():
    for inner_key, value in inner_dict.items():
      for item in value:
        tuples.append((outer_key, inner_key, str(item)))
  return tuples
print(dict_to_tuples(need_tikets))
need_tikets_tuple = dict_to_tuples(need_tikets)

[('207', '30', '1'), ('207', '30', '2'), ('207', '30', '3'), ('207', '30', '4'), ('207', '30', '5'), ('207', '30', '6'), ('207', '30', '7'), ('207', '30', '8'), ('207', '30', '9'), ('207', '30', '10'), ('207', '30', '111'), ('207', '30', '112'), ('207', '30', '113'), ('207', '28', '105'), ('207', '28', '106'), ('207', '28', '109'), ('207', '28', '110'), ('207', '28', '111'), ('207', '28', '112'), ('205', '19', '4'), ('205', '19', '5'), ('205', '19', '6'), ('205', '19', '7'), ('205', '19', '8'), ('205', '19', '9'), ('205', '19', '10'), ('205', '22', '105'), ('205', '22', '106'), ('205', '22', '107'), ('205', '22', '108'), ('205', '22', '109')]


In [28]:
from user import User
from tickets import Tickets

In [12]:
ticket = Tickets(url='https://tickets.ska.ru/view-available-zones/2027')
user = User(proxy=proxies[0], account=user_accounts[0])


In [6]:
async def put_in_tickets_queue(sector_row_place: tuple, semaphore: asyncio.Semaphore, tickets_queue: asyncio.Queue):
    async with semaphore:
        sector, row, place = sector_row_place
        is_place_availible = await ticket.find_place_in_sector(sector, row, place)
        print(is_place_availible, sector_row_place)
        if is_place_availible:
            await tickets_queue.put((is_place_availible, sector, row, place))

tickets_queue = asyncio.Queue(30)

async def main():
    semaphore = asyncio.Semaphore(5)
    tasks = []
    for sector_row_place in need_tikets_tuple:
        task = put_in_tickets_queue(sector_row_place, semaphore, tickets_queue)
        await task

In [13]:
await main()

False ('207', '30', '1')
False ('207', '30', '2')
False ('207', '30', '3')
False ('207', '30', '4')
False ('207', '30', '5')
False ('207', '30', '6')
False ('207', '30', '7')
False ('207', '30', '8')
False ('207', '30', '9')
False ('207', '30', '10')
{'categoryId': '1', 'id': '554042', 'name': 'Сектор 207 Ряд 30 Место 111', 'orderId': '', 'price': '0.00', 'quant': '1'} ('207', '30', '111')
{'categoryId': '1', 'id': '554043', 'name': 'Сектор 207 Ряд 30 Место 112', 'orderId': '', 'price': '0.00', 'quant': '1'} ('207', '30', '112')
{'categoryId': '1', 'id': '554044', 'name': 'Сектор 207 Ряд 30 Место 113', 'orderId': '', 'price': '0.00', 'quant': '1'} ('207', '30', '113')
{'categoryId': '1', 'id': '553961', 'name': 'Сектор 207 Ряд 28 Место 105', 'orderId': '', 'price': '0.00', 'quant': '1'} ('207', '28', '105')
{'categoryId': '1', 'id': '553962', 'name': 'Сектор 207 Ряд 28 Место 106', 'orderId': '', 'price': '0.00', 'quant': '1'} ('207', '28', '106')
False ('207', '28', '109')
{'categoryId

In [24]:

print(tickets_queue)
#print(box)

<Queue maxsize=30 _queue=[({'categoryId': '1', 'id': '553520', 'name': 'Сектор 206 Ряд 30 Место 1', 'orderId': '', 'price': '0.00', 'quant': '1'}, '206', '30', '1'), ({'categoryId': '1', 'id': '553521', 'name': 'Сектор 206 Ряд 30 Место 2', 'orderId': '', 'price': '0.00', 'quant': '1'}, '206', '30', '2'), ({'categoryId': '1', 'id': '553522', 'name': 'Сектор 206 Ряд 30 Место 3', 'orderId': '', 'price': '0.00', 'quant': '1'}, '206', '30', '3'), ({'categoryId': '1', 'id': '553523', 'name': 'Сектор 206 Ряд 30 Место 4', 'orderId': '', 'price': '0.00', 'quant': '1'}, '206', '30', '4'), ({'categoryId': '1', 'id': '553524', 'name': 'Сектор 206 Ряд 30 Место 5', 'orderId': '', 'price': '0.00', 'quant': '1'}, '206', '30', '5'), ({'categoryId': '1', 'id': '553525', 'name': 'Сектор 206 Ряд 30 Место 6', 'orderId': '', 'price': '0.00', 'quant': '1'}, '206', '30', '6'), ({'categoryId': '1', 'id': '553526', 'name': 'Сектор 206 Ряд 30 Место 7', 'orderId': '', 'price': '0.00', 'quant': '1'}, '206', '30', 

In [30]:
async def auth_account(user_accounts, proxies):
    if len(user_accounts) <= 0:
        print('Out of users!')
        return False
    user_accounts = user_accounts.pop()
    if len(proxies) <= 0:
        print('Out of proxies')
        return False
    proxy = proxies.pop()
    user = User(proxy, user_accounts)
    status_code = await user.make_session()
    if status_code != 200:
        return auth_account(user_accounts, proxies)
    auth_status = await user.auth()
    if not auth_status:
        return False
    return user
user_1 = await auth_account(user_accounts, proxies)     


ELifMiI6QNPYMJVshvmDTfJZsXikKJyFBQ4D76J3mR1-jM1_RH4Qiop-3zjAwfQMmhLbId57-8BOZXeG1hrjTA== CSRF token
<ClientResponse(https://tickets.ska.ru/) [200 OK]>
<CIMultiDictProxy('Server': 'nginx', 'Date': 'Sat, 24 Feb 2024 17:01:43 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Set-Cookie': 'SKA_SESSION=tk6clu1anq6kmv6p74cir7tg86; expires=Sun, 25-Feb-2024 17:01:41 GMT; Max-Age=86400; path=/', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Strict-Transport-Security': 'max-age=31536000;', 'Content-Encoding': 'gzip')>
 True self.auth_status


In [31]:
user_2 = await auth_account(user_accounts, proxies)

jAvVdv5QA_Y9MobQz_gtNyB7DvNGP5y2jfS-jRqJ2WPTRp0Pl2VJg2508oCMrkxDYy17phBt5Pf4rdfYSOXgVw== CSRF token
<ClientResponse(https://tickets.ska.ru/) [200 OK]>
<CIMultiDictProxy('Server': 'nginx', 'Date': 'Sat, 24 Feb 2024 17:01:49 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Set-Cookie': 'SKA_SESSION=dpkovu0u8mgt3m59cca89255m0; expires=Sun, 25-Feb-2024 17:01:47 GMT; Max-Age=86400; path=/', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Strict-Transport-Security': 'max-age=31536000;', 'Content-Encoding': 'gzip')>
 True self.auth_status


In [28]:
print(user_1)

In [ ]:
acc = {
        "login": "mikheyev98@gmail.com",
        "password": "Ai5F,;baB:rVf!8"
    }
user_3 = User(proxy="213.139.223.17:8000@XaW2zS:SVgsuG", account=acc)
x = await user_3.make_session()

In [25]:
await user_3.close_session()

In [ ]:
async def monitor_tickets_queue(tickets_queue, semaphore):
    while True:
        # Ожидаем получения 5 билетов из очереди
        tickets_batch = []
        for _ in range(3):
            ticket = await tickets_queue.get()
            tickets_batch.append(ticket)
        
        # Создаем и запускаем задачу для обработки пакета билетов
        async with semaphore:
            await asyncio.create_task(process_tickets_batch(tickets_batch))


async def main():
    tickets_queue = asyncio.Queue()
    # Создаем семафор для ограничения количества одновременно выполняемых задач
    semaphore = asyncio.Semaphore(10)
    
    # Запускаем асинхронную функцию для добавления билетов в очередь
    asyncio.create_task(add_tickets_to_queue(tickets_queue))

    # Запускаем мониторинг очереди
    await monitor_tickets_queue(tickets_queue, semaphore)

In [1]:
from tickets import Tickets

In [2]:
ticket = Tickets('https://tickets.ska.ru/view-available-zones/2062')

In [3]:
all_sectors = await ticket._update_all_free_sectors()

In [4]:
print(all_sectors)

[{'id': '550733', 'name': 202, 'quant': '1', 'zoneId': '-99999', 'zoneTypeId': '100', 'price': [650, 1190], 'url': 'https://tickets.ska.ru/seats-list/2062/550733'}, {'id': '551226', 'name': 203, 'quant': '1', 'zoneId': '-99999', 'zoneTypeId': '100', 'price': [995, 2050], 'url': 'https://tickets.ska.ru/seats-list/2062/551226'}, {'id': '551861', 'name': 204, 'quant': '3', 'zoneId': '-99999', 'zoneTypeId': '100', 'price': [1595, 3190], 'url': 'https://tickets.ska.ru/seats-list/2062/551861'}, {'id': '552377', 'name': 205, 'quant': '2', 'zoneId': '-99999', 'zoneTypeId': '100', 'price': [1695, 3300], 'url': 'https://tickets.ska.ru/seats-list/2062/552377'}, {'id': '554699', 'name': 209, 'quant': '1', 'zoneId': '-99999', 'zoneTypeId': '100', 'price': [595, 1150], 'url': 'https://tickets.ska.ru/seats-list/2062/554699'}, {'id': '557417', 'name': 213, 'quant': '2', 'zoneId': '-99999', 'zoneTypeId': '100', 'price': [1675, 3290], 'url': 'https://tickets.ska.ru/seats-list/2062/557417'}, {'id': '5590

In [5]:
ticket.all_free_sectors

{'202': {'url': 'https://tickets.ska.ru/seats-list/2062/550733'},
 '203': {'url': 'https://tickets.ska.ru/seats-list/2062/551226'},
 '204': {'url': 'https://tickets.ska.ru/seats-list/2062/551861'},
 '205': {'url': 'https://tickets.ska.ru/seats-list/2062/552377'},
 '209': {'url': 'https://tickets.ska.ru/seats-list/2062/554699'},
 '213': {'url': 'https://tickets.ska.ru/seats-list/2062/557417'},
 '215': {'url': 'https://tickets.ska.ru/seats-list/2062/559044'},
 '216': {'url': 'https://tickets.ska.ru/seats-list/2062/559540'},
 '304': {'url': 'https://tickets.ska.ru/seats-list/2062/560617'},
 '305': {'url': 'https://tickets.ska.ru/seats-list/2062/560734'},
 'Ресторан 4 этаж': {'url': 'https://tickets.ska.ru/seats-list/2062/560955'},
 '601': {'url': 'https://tickets.ska.ru/seats-list/2062/563413'},
 '602': {'url': 'https://tickets.ska.ru/seats-list/2062/564117'},
 '603': {'url': 'https://tickets.ska.ru/seats-list/2062/564772'},
 '604': {'url': 'https://tickets.ska.ru/seats-list/2062/565363'}

In [6]:
sector_update = await ticket._update_rows_seats_in_sector('604')

In [11]:
ticket.all_tickets_structure

{'604': {'4': {'3': {'categoryId': '1',
    'id': '565455',
    'name': 'Сектор 604 Ряд 4 Место 3',
    'orderId': '',
    'price': '1590.00',
    'quant': '1'},
   '103': {'categoryId': '1',
    'id': '565470',
    'name': 'Сектор 604 Ряд 4 Место 103',
    'orderId': '',
    'price': '1590.00',
    'quant': '1'},
   '111': {'categoryId': '1',
    'id': '565478',
    'name': 'Сектор 604 Ряд 4 Место 111',
    'orderId': '',
    'price': '2390.00',
    'quant': '1'}},
  '5': {'103': {'categoryId': '1',
    'id': '565497',
    'name': 'Сектор 604 Ряд 5 Место 103',
    'orderId': '',
    'price': '2390.00',
    'quant': '1'}},
  '7': {'111': {'categoryId': '1',
    'id': '565559',
    'name': 'Сектор 604 Ряд 7 Место 111',
    'orderId': '',
    'price': '2290.00',
    'quant': '1'}},
  '8': {'9': {'categoryId': '1',
    'id': '565569',
    'name': 'Сектор 604 Ряд 8 Место 9',
    'orderId': '',
    'price': '2190.00',
    'quant': '1'},
   '104': {'categoryId': '1',
    'id': '565584',
    

In [1]:
from controllerr import create_need_tickets

In [4]:
create_need_tickets()